In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import math
TRAIN_FILE = 'data/traindata.csv'
TEST_FILE = 'data/testdata.csv'
c = 1
k = 2

In [ ]:
df = pd.read_csv(TRAIN_FILE,sep=',',header=None)

In [ ]:
data = np.array(df)

In [ ]:
ind = np.asarray(data[:,0]).astype(int)
comment = data[:,5]

In [ ]:
m = len(comment)
print(m)

In [ ]:
phiY = np.sum(ind)/(4.0*m)  ## y==4
num0 = (1-phiY)*m
num1 = m - num0

In [ ]:
dct = {}
for i in range(0,len(comment)):
    words = set(comment[i].split())
    if(ind[i] == 0):
        for w in words:
            if w in dct:
                dct[w][0] += 1
            else:
                dct[w] = [1,0]
    else:
        for w in words:
            if w in dct:
                dct[w][1] += 1
            else:
                dct[w] = [0,1]

In [ ]:
# log probab with laplacian smoothing 
def p(x,y):
    occur = [0,0]
    if x in dct:
        occur = dct[x]
    if(y==0):
        return math.log((float)(occur[0]+c)/(float)(num0+(c*k)));
    else:
        return math.log((float)(occur[1]+c)/(float)(num1+(c*k)));

In [ ]:
def py4x(x):
    p0=0
    p4=0
    for val in x:
        p0 += p(val,0)
        p4 += p(val,4)
    p0 = math.exp(p0)*(1-phiY)
    p4 = math.exp(p4)*(phiY)
    return p4/(p0+p4);

In [ ]:
#testing
tdf = pd.read_csv(TEST_FILE,sep=',',header=None)
tdata = np.array(tdf)
tind = np.asarray(tdata[:,0]).astype(int)
tcomment = tdata[:,5]

In [ ]:
error = np.zeros(2)
total = 0
cnfmat = np.zeros((2,2)) 
for i in range(0,len(tcomment)):
    if(tind[i] == 2):
        continue;
    total+=1;
    p1 = py4x(set(tcomment[i].split()))
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
    ## confusion matrix
    cnfmat[clss/4][tind[i]/4]+=1

In [ ]:
print(error/total)

In [ ]:
cnfmat

In [ ]:
# part(b) Baseline accuracy
# random guessing
import random
total = 0
error = np.zeros(2)
for i in range(0,len(tcomment)):
    if(tind[i]==2):
        continue;
    total+=1;
    p1 = random.random()
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
print(error/total)
# Theoretically must be 0.5 with very large data set

total = 0
error = np.zeros(2)
for i in range(0,len(tcomment)):
    if(tind[i]==2):
        continue;
    total+=1;
    p1 = phiY
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
print(error/total)
#equal to the num of majority class present.

In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import unicodedata
tknzr = TweetTokenizer(strip_handles=True)
ps = nltk.stem.PorterStemmer()
lmt = nltk.stem.WordNetLemmatizer() 
sn = nltk.stem.SnowballStemmer('english')

In [ ]:
#preprocessing: returns list of words after processing. not set
def process(s):
    # remove twitter handle then digits and punctuation
    s = s.decode('latin-1')
    words = ([sn.stem(word.lower()) for word in tknzr.tokenize(s) if word.isalpha() and word not in stopwords.words('english') and len(word)>2]) 
    return words

In [ ]:
dct = {}
for i in range(0,len(comment)):
    if(i%10000==0):
        print(i)
    words = set(process(comment[i]))
    if(ind[i]==0):
        for w in words:
            if w in dct:
                dct[w][0] += 1;
            else:
                dct[w] = [1,0]
    else:
        for w in words:
            if w in dct:
                dct[w][1] += 1;
            else:
                dct[w] = [0,1]

In [ ]:
stemdct = dct

In [ ]:
error = np.zeros(2)
total = 0
cnfmat = np.zeros((2,2)) 
for i in range(0,len(tcomment)):
    if(tind[i] == 2):
        continue;
    total+=1;
    p1 = py4x(process(tcomment[i]))
    clss = 0
    if(p1 > 0.5):
        clss = 4;
    if(clss == tind[i]):
        error[0]+=1;
    else:
        error[1]+=1;
    ## confusion matrix
    cnfmat[clss/4][tind[i]/4]+=1

In [ ]:
print(error/total)

In [ ]:
cnfmat

In [ ]:
# feature engineering
#1 k-gram 
#2 divide the weight of each word in a sentence ( partial tf-idf? )

In [ ]:
c = comment[0:100]

In [ ]:
len(c)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.naive_bayes import GaussianNB
import time
clf = GaussianNB()
sp = SelectPercentile(chi2, percentile=0.1)    
vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words=stopwords.words('english'),decode_error='ignore', encoding='latin-1')

In [ ]:
X = vectorizer.fit_transform(comment)

In [ ]:
opt = True

In [ ]:
X.get_shape()

In [ ]:
X = X_old

In [ ]:
if opt:
## Selecting percentiles
    X_new = sp.fit_transform(X, ind)

In [ ]:
X_old = X
X = X_new

In [ ]:
X_old.get_shape()

In [ ]:
X.get_shape()

In [ ]:
time0 = time.time()
BS = 1000
for i in range(0,len(comment)/BS):
    print(i)
    clf.partial_fit(X[i*BS:(i+1)*BS].toarray(),ind[i*BS:(i+1)*BS],classes=[0,4])
time1 = time.time()
print(time1-time0)

In [ ]:
# Testing using 
tX = vectorizer.transform(tcomment)
# if opt:
#     tX = sp.transform(tX)
print(tX.get_shape())
acc = 0
total = 0
for i in range(0,len(tcomment)):
    if tind[i] == 2:
        continue;
    total+=1;
    res = clf.predict(tX[i].toarray())[0]
#     print(res)
    if(res == tind[i]):
        acc +=1

In [ ]:
print(float(acc)/float(total))

In [ ]:

che.toarray().shape

In [ ]:
clf.predict(vectorizer.transform(['i am saksham','i am not a good']).toarray())

In [ ]:
#tf-idf
dct = {}
for i in range(0,len(comment)):
    if(i%1000==0):
        print(i)
    words = process(comment[i]) ## net non unique also here
    if(ind[i]==0):
        for w in words:
            if w in dct:
                dct[w][0][0] += 1;
            else:
                dct[w] = [[1,0],[0,0]]
        for w in set(words):
            dct[w][0][1]+=1;
    else:
        for w in words:
            if w in dct:
                dct[w][1][0] += 1;
            else:
                dct[w] = [[0,0],[0,1]]
        for w in set(words):
            dct[w][1][1]+=1;


In [ ]:
# print(dct)
tfdct = dct
dct = {}
for w in tfdct:
    d0 = 0.0
    d1 = 0.0
    if(tfdct[w][0][0]!=0):
        d0 = (float)(tfdct[w][0][0])/(float)(tfdct[w][0][1])
    if(tfdct[w][1][0]!=0):
        d1 = (float)(tfdct[w][1][0])/(float)(tfdct[w][1][1])
    dct[w] = [d0,d1]

In [ ]:
dct